In [12]:
import torch
import pandas as pd
import numpy as np
import preprocess as p
import pickle as pkl
from train import trainer, tester, priorityChecker, predict
from dataset import CharData
from modules import FinTransformer
from encoder import EncFinTransformer

In [4]:
data = CharData()

In rank normalize


  0%|          | 0/780 [00:00<?, ?it/s]f:\Raina\Study\JHU\Research\Dr.Gupta\LLM\v1\dataset.py:68: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cross_slice.loc[cross_slice['DATE'] == date] = cross_slice.fillna(0) + omitted_mask * cross_slice.median()
  0%|          | 1/780 [00:00<11:20,  1.15it/s]f:\Raina\Study\JHU\Research\Dr.Gupta\LLM\v1\dataset.py:68: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cross_slice.loc[cross_slice['DATE'] == date] = cross_slice.fillna(0) + omitted_mask * cross_slice.median()
  0%|          | 2/780 [00:01<07:09,  1.81it/s

In joinrets
In Loaders


100%|██████████| 360/360 [00:04<00:00, 78.97it/s]


In [6]:
batch_size = 512
embed_size = 16
inner_dim = 16
num_characteristics = 94
heads = 8
repeats = 3
dropout = 0.2
batch_size = batch_size
num_companies = batch_size

testloader = torch.load("testloader.pt")
test_model = FinTransformer(embed_size, inner_dim, num_companies, num_characteristics, heads, repeats, dropout)
test_model.load_state_dict(torch.load("./checkpoint/best_model.pt"))

<All keys matched successfully>

In [7]:
outputs = predict(model=test_model, testloader=testloader, num_companies=num_companies, num_characteristics=num_characteristics)

  0%|          | 0/4916 [00:00<?, ?it/s]

100%|██████████| 4916/4916 [03:47<00:00, 21.58it/s]


In [8]:
output, target = outputs[0], outputs[1]

In [9]:
output = torch.cat(output, dim = 0)
target = torch.cat(target, dim = 0)

In [10]:
torch.save(output, "preds.pt")
torch.save(target, "targets.pt")

In [13]:


testdata = pd.read_csv("test.csv")

In [14]:
testdata.shape

(2516889, 97)

In [15]:
output.view(-1).shape

torch.Size([2516992])

In [16]:
testdata.columns

Index(['absacc', 'acc', 'age', 'agr', 'bm', 'bm_ia', 'cashdebt', 'cashpr',
       'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chpmia',
       'convind', 'currat', 'depr', 'divi', 'divo', 'dy', 'egr', 'ep', 'gma',
       'grcapx', 'grltnoa', 'herf', 'hire', 'invest', 'lev', 'lgr', 'mve_ia',
       'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
       'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect',
       'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps', 'quick', 'rd',
       'rd_mve', 'rd_sale', 'realestate', 'roic', 'salecash', 'saleinv',
       'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'tang', 'tb',
       'aeavol', 'cash', 'chtx', 'cinvest', 'ear', 'ms', 'nincr', 'roaq',
       'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf', 'baspread', 'beta',
       'betasq', 'chmom', 'dolvol', 'idiovol', 'ill', 'indmom', 'maxret',
       'mom12m', 'mom1m', 'mom36m', 'mom6m', 'mvel1', 'pricedelay', 'retvol',
       'std_dolvol', 'std_turn', '

In [17]:
import pandas as pd
testdata = pd.read_csv("FinalPredictions.csv")

In [18]:
testdata["ret-rf"][:10]

0   -0.183182
1    0.022790
2   -0.106518
3    2.085095
4    0.036765
5   -0.149913
6    1.287183
7   -0.327159
8    2.206384
9   -0.549023
Name: ret-rf, dtype: float64

In [19]:
testdata["predictions"][:10]

0   -0.177973
1    0.020929
2   -0.104212
3    2.074203
4    0.034352
5   -0.146083
6    1.294620
7   -0.313496
8    2.200506
9   -0.511939
Name: predictions, dtype: float64

In [20]:
import joblib

scaler = joblib.load("ReturnScaler.obj")
excess = scaler.inverse_transform(testdata["ret-rf"])
excess_pred = scaler.inverse_transform(testdata["predictions"])

ValueError: Expected 2D array, got 1D array instead:
array=[-0.1831818   0.02278974 -0.10651751 ... -0.05556056 -1.0487953
 -0.26248229].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
rets = np.array(testdata["ret-rf"])
excess = scaler.inverse_transform(rets.reshape(-1,1))

In [22]:
preds = np.array(testdata["predictions"])
expr = scaler.inverse_transform(preds.reshape(-1,1))

In [23]:
expr[:100]

array([[ -2.36279328],
       [  1.07017873],
       [ -1.08970843],
       [ 36.5089999 ],
       [  1.30186493],
       [ -1.8123953 ],
       [ 23.05366213],
       [ -4.70187743],
       [ 38.68895089],
       [ -8.12694139],
       [-13.73357972],
       [ -2.40378202],
       [ 13.00977273],
       [-29.78058908],
       [  2.84906086],
       [ 64.02821358],
       [  8.13501013],
       [ 11.5314973 ],
       [ -6.86191479],
       [ 45.80600197],
       [  0.97138829],
       [-15.06403625],
       [ 11.2797164 ],
       [ -0.42456915],
       [ -3.15082161],
       [ -2.2940559 ],
       [ -9.06738613],
       [  6.96786454],
       [ -2.23608343],
       [  2.88971422],
       [ -8.02997083],
       [ -1.06947279],
       [  9.21335142],
       [ -0.7904278 ],
       [132.05617844],
       [  1.69324496],
       [ -3.1923597 ],
       [  9.43832499],
       [ -0.48083903],
       [ -0.32277783],
       [ -4.78493819],
       [ -8.32343281],
       [  1.97679488],
       [  2

In [24]:
excess[:100]

array([[ -2.4527],
       [  1.1023],
       [ -1.1295],
       [ 36.697 ],
       [  1.3435],
       [ -1.8785],
       [ 22.9253],
       [ -4.9377],
       [ 38.7904],
       [ -8.767 ],
       [-14.1331],
       [ -2.4956],
       [ 13.1521],
       [-28.9114],
       [  2.9511],
       [ 64.4748],
       [  8.195 ],
       [ 11.6258],
       [ -7.3167],
       [ 46.3267],
       [  0.9994],
       [-15.2336],
       [ 11.3673],
       [ -0.4445],
       [ -3.2812],
       [ -2.3808],
       [ -9.8638],
       [  7.0674],
       [ -2.3202],
       [  2.9933],
       [ -8.6541],
       [ -1.1086],
       [  9.249 ],
       [ -0.8208],
       [125.9758],
       [  1.7506],
       [ -3.3251],
       [  9.4782],
       [ -0.5023],
       [ -0.34  ],
       [ -5.0275],
       [ -8.9967],
       [  2.0453],
       [  2.1476],
       [  4.2472],
       [-10.1439],
       [ -1.2209],
       [ -8.1426],
       [-12.6207],
       [ 16.3267],
       [-12.84  ],
       [  1.4782],
       [ -0.